In [4]:
import rasterio
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import datetime
import glob as glob
import datetime
import sys
import os
import geopandas as gp
import rasterio
# import seaborn as sns
%matplotlib inline

epoch = datetime.datetime.utcfromtimestamp(0)
def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

In [5]:
gcs = '/mnt'

In [7]:
fhs = glob.glob(gcs + '/rain_idw_persiann/*.tif')
imageorder = np.array(
    [int(fh.split('_')[-1].split('.')[0]) for fh in fhs]
)
fhs = [fhs[i] for i in imageorder.argsort()]
dates = pd.date_range('1980-01-01', '2014-01-01')[:-1]
points_gdf = gp.read_file(gcs + '/sesync2018_points.geojson')
points = [item for item in zip(points_gdf['longitude'].values, points_gdf['latitude'].values)]
data = np.zeros((len(dates), len(points)))

# for each day, extract rainfall from each point in points
# save the data as an nxm array, where n is the number of days
# and m is the number of sites
for i,f in enumerate(fhs):
    with rasterio.open(f) as src:
        test = np.array([val for val in src.sample(points)]).flatten()
    data[i,:] = test

# time series of rainfall at each site
df_daily = pd.DataFrame(data, index=dates)
todrop = [col for col in df_daily.columns if (df_daily[col].sum()<=0)|(~np.isfinite(df_daily[col].sum()))]
df_daily.drop(todrop, inplace=True, axis=1)
# df_daily.to_csv('./data/sesync2018_points_rainIdw.csv')

/usr/lib/python2.7/dist-packages/rasterio/sample.py:9: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  data = read(indexes, window=window, masked=False, boundless=True)
